In [166]:
import numpy as np
import os
import time
import corner
import astropy.io.fits as pyfits

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
%matplotlib inline

import astropy.units as u
from astropy.wcs import WCS
from lenstronomy.LensModel.lens_model import LensModel
from lenstronomy.LensModel.Solver.lens_equation_solver import LensEquationSolver
from lenstronomy.LightModel.light_model import LightModel
from lenstronomy.PointSource.point_source import PointSource
from lenstronomy.ImSim.image_model import ImageModel
import lenstronomy.Util.param_util as param_util
import lenstronomy.Util.simulation_util as sim_util
import lenstronomy.Util.image_util as image_util
from lenstronomy.Util import kernel_util
from lenstronomy.Data.imaging_data import ImageData
from lenstronomy.Data.psf import PSF
import matplotlib.pylab as pylab


plt.style.use("science")
params = {'legend.fontsize': 'x-large',
#           'figure.figsize': (10, 10),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
pylab.rcParams.update(params)


In [167]:
SNR = 5.098 # known from original fits file
sky_zero_point = -21.1 # mag
photflam = 6.9548454E-20 # inverse sensitivity ergs/cm2/Ang/electron
galfit_data = fits.open("../data/blocks1.fits")
im = galfit_data[3].data
wcs= WCS(galfit_data[1].header)
psf = fits.open("../data/rxj1131_psf1.fits")[0].data
psf2 = fits.open("../data/tiny_psf00_psf.fits")[0].data
im[im < 0] = 0
image_principales = np.loadtxt("../data/images_positions.txt")[:4] ## B, A, C, D, G, S
f_vega = 8.60e-10 
image_principales

array([[ 47.63010754, 114.14992219],
       [ 48.31652521,  84.48420784],
       [ 63.11519005,  56.54091825],
       [126.24860758, 106.42748217]])

In [169]:
# coordinate system in arcseconds, centered
N, M = im.shape
y = np.arange(N) #- N//2. + 0.5 * ((N + 1) % 2)
x = np.arange(M) #- M//2. + 0.5 * ((M + 1) % 2)
x, y = np.meshgrid(x, y)
lens_sky = wcs.pixel_to_world(x.ravel(), y.ravel())
x_center = (lens_sky.ra[M//2]).to(u.arcsec)
y_center = (lens_sky.dec[M*N//2]).to(u.arcsec)

x_image = image_principales[:, 0][[1, 0, 2, 3]] # A, B, C, D
y_image = image_principales[:, 1][[1, 0, 2, 3]]
positions = wcs.pixel_to_world(x_image, y_image)
x_image = (positions.ra - x_center).to(u.arcsec).value
y_image = (positions.dec - y_center).to(u.arcsec).value
image_position = pd.DataFrame(np.column_stack([x_image, y_image]), columns=["theta_x", "theta_y"])
image_position

,theta_x,theta_y
0,1.871753,-0.620745
1,1.899809,0.565885
2,1.265421,-1.738511
3,-1.321674,0.256811


In [170]:
x_center.to(u.hourangle)
# y_center.to(u.deg)

<Longitude 11.53096118 hourangle>

In [171]:
# time relative to first image (A) in order AB, Ac, and AD
delta_t = pd.DataFrame(np.array([[0.7, -0.4, 91.72], [1.4, 2, 1.5]]).T, columns=["delta_t", "sigma"])
delta_t.to_csv("../data/time_delays.csv")
delta_t

,delta_t,sigma
0,0.70,1.4
1,-0.40,2.0
2,91.72,1.5


In [172]:
x_image_ = (lens_sky.ra - lens_sky.ra.min()).to(u.arcsec).value
y_image_ = (lens_sky.dec - lens_sky.dec.min()).to(u.arcsec).value
y_image_.max()

7.960418430753435

In [173]:
# data specifics for the lens image
sigma_bkg = .05  #  background noise per pixel (Gaussian)
exp_time = 1.  #  exposure time (arbitrary units, flux per pixel is in units #photons/exp_time unit)
delta_pix_x = np.abs(lens_sky.ra[0] - lens_sky.ra[1]).to(u.arcsec).value
delta_pix_y = np.abs(lens_sky.dec[0] - lens_sky.dec[M]).to(u.arcsec).value
data_class = ImageData(im, exp_time, sigma_bkg, ra_at_xy_0=0, dec_at_xy_0=0,
                       transform_pix2angle=np.array([[delta_pix_x, 0], [0, delta_pix_y]]))

In [174]:
x = lens_sky.ra.to(u.arcsec) - x_center
y = lens_sky.dec.to(u.arcsec) - y_center
grid = np.column_stack([x, y])
ee = 0.611 # encircled energy, source https://www.stsci.edu/hst/instrumentation/acs/data-analysis/aperture-corrections
# collect 4 pixels closest to the image position (correspond to 0.1 arcsec since pixelscale is 0.05 arcsec)
f = []
A = []
for i in range(4):
    pos = image_position.to_numpy()[i]
    diff = np.square(pos - grid.value)
    dist = np.einsum("ij -> i", diff)
    indexes = np.argpartition(dist, kth=4)[:4]
    A.append(np.sum(im.ravel()[indexes]))
    f.append(photflam * np.sum(im.ravel()[indexes])/ee) # see e.g. https://www.stsci.edu/hst/instrumentation/acs/data-analysis/zeropoints
mag = np.array([-2.5 * np.log10(_f/f_vega) for _f in f])
mag

array([18.04445631, 18.0809676 , 18.98064053, 20.35414033])

In [175]:
# kwargs_ps = [{'ra_image': x_image, 'dec_image': y_image,}]
#                            'point_amp': np.abs(mag)*1000}]  
point_source_list = ['LENSED_POSITION']
# point_source_class = PointSource(point_source_type_list=point_source_list, fixed_magnification_list=[False])

In [176]:
kwargs_psf = {'psf_type': "PIXEL", "kernel_point_source":psf, "point_source_supersampling_factor":1}

In [177]:
# lens model choicers
fixed_lens = []
kwargs_lens_init = []
kwargs_lens_sigma = []
kwargs_lower_lens = []
kwargs_upper_lens = []

fixed_lens.append({}) 
kwargs_lens_init.append({'theta_E': 1.6, 'gamma': 2, 'center_x': 0.0, 'center_y': 0, 'e1': 0, 'e2': 0.})
#kwargs_lens_init.append(kwargs_spemd)
kwargs_lens_sigma.append({'theta_E': .2, 'e1': 0.1, 'e2': 0.1, 'gamma': 0.1, 'center_x': 0.01, 'center_y': 0.01})
kwargs_lower_lens.append({'theta_E': 0.01, 'e1': -0.5, 'e2': -0.5, 'gamma': 1.5, 'center_x': -10, 'center_y': -10})
kwargs_upper_lens.append({'theta_E': 10, 'e1': 0.5, 'e2': 0.5, 'gamma': 2.5, 'center_x': 10, 'center_y': 10})

fixed_lens.append({'ra_0': 0, 'dec_0': 0})
kwargs_lens_init.append({'gamma1': 0, 'gamma2': 0})
#kwargs_lens_init.append(kwargs_shear)
kwargs_lens_sigma.append({'gamma1': 0.1, 'gamma2': 0.1})
kwargs_lower_lens.append({'gamma1': -0.5, 'gamma2': -0.5})
kwargs_upper_lens.append({'gamma1': 0.5, 'gamma2': 0.5})

lens_params = [kwargs_lens_init, kwargs_lens_sigma, fixed_lens, kwargs_lower_lens, kwargs_upper_lens]

# lens light model choices
# fixed_lens_light = []
# kwargs_lens_light_init = []
# kwargs_lens_light_sigma = []
# kwargs_lower_lens_light = []
# kwargs_upper_lens_light = []

# fixed_lens_light.append({})
# kwargs_lens_light_init.append({'R_sersic': 0.5, 'n_sersic': 1, 'e1': 0, 'e2': 0., 'center_x': 0, 'center_y': 0})
# #kwargs_lens_light_init.append(kwargs_sersic_lens)
# kwargs_lens_light_sigma.append({'n_sersic': 0.5, 'R_sersic': 0.1, 'e1': 0.1, 'e2': 0.1, 'center_x': 0.1, 'center_y': 0.1})
# kwargs_lower_lens_light.append({'e1': -0.5, 'e2': -0.5, 'R_sersic': 0.01, 'n_sersic': 0.5, 'center_x': -10, 'center_y': -10})
# kwargs_upper_lens_light.append({'e1': 0.5, 'e2': 0.5, 'R_sersic': 10, 'n_sersic': 8, 'center_x': 10, 'center_y': 10})

# lens_light_params = [kwargs_lens_light_init, kwargs_lens_light_sigma, fixed_lens_light, kwargs_lower_lens_light, kwargs_upper_lens_light]


fixed_source = []
kwargs_source_init = []
kwargs_source_sigma = []
kwargs_lower_source = []
kwargs_upper_source = []

fixed_source.append({})
kwargs_source_init.append({'R_sersic': 0.1, 'n_sersic': 1, 'e1': 0, 'e2': 0., 'center_x': 0, 'center_y': 0})
#kwargs_source_init.append(kwargs_sersic_source)
kwargs_source_sigma.append({'n_sersic': 0.5, 'R_sersic': 0.05, 'e1': 0.1, 'e2': 0.1, 'center_x': 0.1, 'center_y': 0.1})
kwargs_lower_source.append({'e1': -0.5, 'e2': -0.5, 'R_sersic': 0.001, 'n_sersic': .5, 'center_x': -10, 'center_y': -10})
kwargs_upper_source.append({'e1': 0.5, 'e2': 0.5, 'R_sersic': 10, 'n_sersic': 5., 'center_x': 10, 'center_y': 10})

source_params = [kwargs_source_init, kwargs_source_sigma, fixed_source, kwargs_lower_source, kwargs_upper_source]


fixed_ps = [{}]
kwargs_ps_init = kwargs_ps
kwargs_ps_sigma = [{'ra_image': plate_scale/2 * np.ones(len(x_image)), 'dec_image': plate_scale/2 * np.ones(len(x_image))}]
kwargs_lower_ps = [{'ra_image': -10 * np.ones(len(x_image)), 'dec_image': -10 * np.ones(len(y_image))}]
kwargs_upper_ps = [{'ra_image': 10 * np.ones(len(x_image)), 'dec_image': 10 * np.ones(len(y_image))}]

fixed_cosmo = {}
kwargs_cosmo_init = {'D_dt': 5000}
kwargs_cosmo_sigma = {'D_dt': 10000}
kwargs_lower_cosmo = {'D_dt': 0}
kwargs_upper_cosmo = {'D_dt': 10000}
cosmo_params = [kwargs_cosmo_init, kwargs_cosmo_sigma, fixed_cosmo, kwargs_lower_cosmo, kwargs_upper_cosmo]

ps_params = [kwargs_ps_init, kwargs_ps_sigma, fixed_ps, kwargs_lower_ps, kwargs_upper_ps]
source_model_list = ['SERSIC_ELLIPSE']
lens_model_list = ['SPEP', 'SHEAR']

kwargs_params = {'lens_model': lens_params,
                'source_model': source_params,
#                 'lens_light_model': lens_light_params,
                'point_source_model': ps_params,
                'special': cosmo_params}

kwargs_model = {'lens_model_list': lens_model_list, 
#                  'lens_light_model_list': lens_light_model_list,
                 'source_light_model_list': source_model_list,
                'point_source_model_list': point_source_list
                 }



In [178]:
# numerical options and fitting sequences


num_source_model = len(source_model_list)

kwargs_constraints = {'joint_source_with_point_source': [[0, 0]],
                      'num_point_source_list': [4],
                      'solver_type': 'PROFILE_SHEAR',  # 'PROFILE', 'PROFILE_SHEAR', 'ELLIPSE', 'CENTER'
                      'Ddt_sampling': True,
                              }

kwargs_likelihood = {'check_bounds': True,
                     'force_no_add_image': False,
                     'source_marg': False,
                     'image_position_uncertainty': 0.004,
                     'check_matched_source_position': True,
                     'source_position_tolerance': 0.001,
                     'time_delay_likelihood': True,
                             }
kwargs_numerics = {'supersampling_factor': 1}
image_band = [kwargs_data, kwargs_psf, kwargs_numerics]
multi_band_list = [image_band]
kwargs_data_joint = {'multi_band_list': multi_band_list, 'multi_band_type': 'multi-linear',
                    'time_delays_measured': delta_t["delta_t"].to_numpy(),
                    'time_delays_uncertainties': delta_t["sigma"].to_numpy(),}

from lenstronomy.Workflow.fitting_sequence import FittingSequence

# mpi = False  # MPI possible, but not supported through that notebook.

# from lenstronomy.Workflow.fitting_sequence import FittingSequence
fitting_seq = FittingSequence(kwargs_data_joint, kwargs_model, kwargs_constraints, kwargs_likelihood, kwargs_params)

fitting_kwargs_list = [['PSO', {'sigma_scale': .1, 'n_particles': 200, 'n_iterations': 200}],
        ['MCMC', {'n_burn': 100, 'n_run': 100, 'walkerRatio': 10, 'sigma_scale': .1}]
]

start_time = time.time()
chain_list = fitting_seq.fit_sequence(fitting_kwargs_list)
kwargs_result = fitting_seq.best_fit()
end_time = time.time()
print(end_time - start_time, 'total time needed for computation')
print('============ CONGRATULATION, YOUR JOB WAS SUCCESSFUL ================ ')

/home/alexandre/anaconda3/lib/python3.8/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/home/alexandre/anaconda3/lib/python3.8/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


Computing the PSO ...
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
Max iteration reached! Stopping.
-1.0636600413327822e-05 reduced X^2 of best position
-0.200063817174283 logL
37618.0 effective number of data points
[{'theta_E': 2.3658337885077687, 'gamma': 1.7669083334267959, 'e1': 0.07575576177175901, 'e2': -0.038662222749004375, 'center_x': -1.2734779016750457, 'center_y': 0.24343630356538107}, {'gamma1': 0.014772281085013907, 'gamma2': 0.01149497354007705, 'ra_0': 0, 'dec_0': 0}] lens result
[{'amp': 1, 'R_sersic': 0.09810574027537357, 'n_sersic': 1.1374566857063526, 'e1': 0.005923657558319148, 'e2': -0.016399631399352316, 'center_x': -0.44848470288804904, 'center_y': 0.012800688920454446}] source result
[] lens light result
[{'ra_image': array([ 1.86759921,  2.0074439 ,  1.305315  , -1.31964725]), 'dec_image': array([-0.61524353,  0.5049699 , -1.74157249,  0.25673262]), 'point_amp': [1, 1, 1, 1]}] point source result
{'D_dt': 861.0073858612643} special p

  0%|          | 0/200 [00:00<?, ?it/s]/home/alexandre/anaconda3/lib/python3.8/site-packages/emcee/moves/red_blue.py:99: RuntimeWarning: invalid value encountered in double_scalars
  lnpdiff = f + nlp - state.log_prob[j]
 17%|█▋        | 34/200 [01:16<06:13,  2.25s/it]/home/alexandre/anaconda3/lib/python3.8/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The number of calls to function has reached maxfev = 1400.
  warnings.warn(msg, RuntimeWarning)
100%|██████████| 200/200 [07:09<00:00,  2.15s/it]

Computing the MCMC...
Number of walkers =  160
Burn-in iterations:  100
Sampling iterations (in current run): 200
431.69836473464966 time taken for MCMC sampling
1102.2629611492157 total time needed for computation
============ CONGRATULATION, YOUR JOB WAS SUCCESSFUL ================ 


In [183]:
from lenstronomy.Plots import chain_plot
from lenstronomy.Plots.model_plot import ModelPlot
import corner

modelPlot = ModelPlot(multi_band_list, kwargs_model, kwargs_result, arrow_size=0.02, cmap_string="gist_heat")
    
f, axes = modelPlot.plot_main()
f.show()
f, axes = modelPlot.plot_separate()
f.show()
f, axes = modelPlot.plot_subtract_from_data_all()
f.show()


ImportError: cannot import name 'quantile' from 'corner.corner' (/home/alexandre/anaconda3/lib/python3.8/site-packages/corner/corner.py)